In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
df = spark.read.option("multiline", "true").json("Files/bing_latest_news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 82061ec5-ad68-4ba0-b71b-0b7f7e6415f9)

In [5]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["organic_results"]).alias("json_object"))

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 7, Finished, Available, Finished)

In [7]:
display(df_exploded)


StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1537bfd3-6e95-45a8-9297-a89e4076cd76)

In [8]:
df=df_exploded.select("json_object.*")

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 10, Finished, Available, Finished)

In [9]:
display(df)

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6e775d53-f427-4091-9eaf-b4cb80ff6944)

In [11]:
from pyspark.sql import functions as F
from datetime import datetime, timedelta
import re
 
# -----------------------------
# Step 1: UDF to convert relative date
# -----------------------------
def convert_relative_date(val):
    if val is None or str(val).strip() == "":
        return None
    val_str = str(val).strip().lower()
    now = datetime.now()
 
    m = re.match(r"^(\d+)([dh])$", val_str)
    if m:
        num, unit = int(m.group(1)), m.group(2)
        if unit == "d":
            return now - timedelta(days=num)
        elif unit == "h":
            return now - timedelta(hours=num)
 
    # Try parsing as absolute date
    try:
        return datetime.strptime(val_str, "%Y-%m-%d")
    except:
        return None
 
 
convert_relative_date_udf = F.udf(convert_relative_date, "timestamp")
df = df.withColumn("date_clean", convert_relative_date_udf(F.col("date")))

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 13, Finished, Available, Finished)

In [12]:
display(df)

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c146b1d9-ff37-4c52-b91a-09566d9a4f9b)

In [13]:
df = df.drop("date","favicon")

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 15, Finished, Available, Finished)

In [14]:
df = df.withColumn(
    "thumbnail",
    F.when(F.col("thumbnail").isNull(), F.lit("https://www.bing.com/th?id=OVFT.placeholder&pid=News"))
    .otherwise(F.col("thumbnail"))
)

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 16, Finished, Available, Finished)

In [15]:
from pyspark.sql.functions import col, year, month, dayofmonth, date_format, hour, to_date
 
df = df.withColumn("date_only", F.to_date(F.col("date_clean")))
df = df.withColumn("year", F.year(F.col("date_clean")))
df = df.withColumn("month", F.month(F.col("date_clean")))
df = df.withColumn("day", F.dayofmonth(F.col("date_clean")))
df = df.withColumn("weekday", F.date_format(F.col("date_clean"), "EEEE"))
df = df.withColumn("hour", F.hour(F.col("date_clean")))

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 17, Finished, Available, Finished)

In [16]:
display(df)

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c4cf9f66-ed15-4036-a9a9-6f2882197bf2)

In [17]:
df.write.format("delta").mode("overwrite").saveAsTable("tbl_latest_news")

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 19, Finished, Available, Finished)

Incremental Load

In [19]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F
 
# Path on your workspace / cluster where Delta files are stored
#delta_path = "abfss://38f83567-163a-49e1-866b-5a801baf7318@onelake.dfs.fabric.microsoft.com/a70173c4-1f34-4d8a-94f1-2e274b2ecc98/Tables/tbl_latest_news" 
delta_path = "abfss://ProjectTFDataEngineering@onelake.dfs.fabric.microsoft.com/LakhehouseTF_DataEngineering.Lakehouse/Tables/tbl_latest_news"
 
# Table name registered in the metastore
table_name = "tbl_latest_news"
 
# -----------------------------
# Step 1: Filter out invalid rows
# -----------------------------
df = df.filter(F.col("link").isNotNull())  # remove rows with null link
 
# -----------------------------
# Step 2: Check if Delta table exists
# -----------------------------
if DeltaTable.isDeltaTable(spark, delta_path):
    # Table exists → do MERGE
    deltaTable = DeltaTable.forPath(spark, delta_path)
 
    # Match schema (avoid mismatches)
    target_cols = set(deltaTable.toDF().columns)
    source_cols = set(df.columns)
    common_cols = list(target_cols.intersection(source_cols))
   
    # Columns to update on match (exclude PK)
    update_dict = {c: f"src.{c}" for c in common_cols if c != "link"}
   
    # Columns to insert on no match (include PK)
    insert_dict = {c: f"src.{c}" for c in common_cols}
 
    # Perform merge
    (deltaTable.alias("tgt")
        .merge(df.alias("src"), "tgt.link = src.link")
        .whenMatchedUpdate(set=update_dict)
        .whenNotMatchedInsert(values=insert_dict)
        .execute()
    )
    print("✅ Incremental load merged into tbl_latest_news")
 
else:
    # First load → write dataframe as Delta table
    (df.write
        .format("delta")
        .mode("overwrite")
        .option("path", delta_path)   # physical path
        .saveAsTable(table_name)      # register in metastore
    )
    print("✅ First load created tbl_latest_news")

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 21, Finished, Available, Finished)

✅ Incremental load merged into tbl_latest_news


In [20]:
%%sql
select * from tbl_latest_news

StatementMeta(, 36d6da6d-bca8-40dc-9a98-8805802f670b, 22, Finished, Available, Finished)

<Spark SQL result set with 44 rows and 13 fields>